In [2]:
### 3. Install dependencies ###

!pip install tesseract-ocr
!pip install libtesseract-dev

!pip install pytesseract
!pip install Pillow
!pip install easyocr
!pip install boto3

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   ----------- ---------------------------- 0.8/2.7 MB 6.7 MB/s eta 0:00:01
   ---------------------------------- ----- 2.4/2.7 MB 7.5 MB/s eta 0:00:01
   ---------------------------------------- 2.7/2.7 MB 7.5 MB/s  0:00:00
Failed to build tesseract-ocr


  error: subprocess-exited-with-error
  
  × Building wheel for tesseract-ocr (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [39 lines of output]
      C:\Users\Bd Calling\AppData\Local\Temp\pip-build-env-5u_jtl3h\overlay\Lib\site-packages\setuptools\_distutils\dist.py:289: UserWarning: Unknown distribution option: 'tests_require'
        warnings.warn(msg)
      C:\Users\Bd Calling\AppData\Local\Temp\pip-build-env-5u_jtl3h\overlay\Lib\site-packages\setuptools\dist.py:599: SetuptoolsDeprecationWarning: Invalid dash-separated key 'description-file' in 'metadata' (setup.cfg), please use the underscore name 'description_file' instead.
      !!
      
              ********************************************************************************
              Usage of dash-separated 'description-file' will not be supported in future
              versions. Please use the underscore name 'description_file' instead.
              (Affected: tesseract-ocr).
      
          

  Using cached easyocr-1.7.2-py3-none-any.whl.metadata (10 kB)
  Using cached torch-2.9.0-cp310-cp310-win_amd64.whl.metadata (30 kB)
  Using cached torchvision-0.24.0-cp310-cp310-win_amd64.whl.metadata (5.9 kB)
  Using cached opencv_python_headless-4.12.0.88-cp37-abi3-win_amd64.whl.metadata (20 kB)
  Using cached scikit_image-0.25.2-cp310-cp310-win_amd64.whl.metadata (14 kB)
  Using cached pyyaml-6.0.3-cp310-cp310-win_amd64.whl.metadata (2.4 kB)
  Using cached shapely-2.1.2-cp310-cp310-win_amd64.whl.metadata (7.1 kB)
  Using cached ninja-1.13.0-py3-none-win_amd64.whl.metadata (5.1 kB)
  Using cached filelock-3.20.0-py3-none-any.whl.metadata (2.1 kB)
  Using cached networkx-3.4.2-py3-none-any.whl.metadata (6.3 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached fsspec-2025.9.0-py3-none-any.whl.metadata (10 kB)
  Using cached markupsafe-3.0.3-cp310-cp310-win_amd64.whl.metadata (2.8 kB)
  Using cached imageio-2.37.0-py3-none-any.whl.metadata (5.2 kB)
  Using 

In [ ]:
### 4. Text detection ###

import pytesseract
from PIL import Image
from easyocr import Reader
import boto3


reader = Reader(['en'])

access_key = None
secret_access_key = None

textract_client = boto3.client('textract',
                               aws_access_key_id=access_key,
                               aws_secret_access_key = secret_access_key,
                               region_name='us-east-1')


def read_text_tesseract(image_path):

  text = pytesseract.image_to_string(Image.open(image_path), lang='eng')
  return text

def read_text_easyocr(image_path):
  text = ''
  results = reader.readtext(Image.open(image_path))
  for result in results:
    text = text + result[1] +  ' '

  text = text[:-1]
  return text

def read_text_textract(image_path):

  with open(image_path, 'rb') as im:
    response = textract_client.detect_document_text(Document={'Bytes':im.read()})

  text = ''
  for item in response['Blocks']:
    if item['BlockType'] == 'LINE':
      text = text + item['Text'] + ' '

  text = text[:-1]

  return text

In [ ]:
### 5. Compare performances ###

import os


def jaccard_similarity(sentence1, sentence2):
    # Tokenize sentences into sets of words
    set1 = set(sentence1.lower().split())
    set2 = set(sentence2.lower().split())

    # Calculate Jaccard similarity
    intersection_size = len(set1.intersection(set2))
    union_size = len(set1.union(set2))

    # Avoid division by zero if both sets are empty
    similarity = intersection_size / union_size if union_size != 0 else 0.0

    return similarity


score_tesseract = 0
score_easyocr = 0
score_textract = 0
for image_path_ in os.listdir('/content/data'):
  image_path = os.path.join('/content/data', image_path_)

  gt = image_path[:-4].replace('_', ' ').lower()

  score_tesseract += jaccard_similarity(gt, read_text_tesseract(image_path).lower().replace('\n', '').replace('!', '').replace('?', '').replace('.', ''))
  score_easyocr += jaccard_similarity(gt, read_text_easyocr(image_path).lower().replace('\n', '').replace('!', '').replace('?', '').replace('.', ''))
  score_textract += jaccard_similarity(gt, read_text_textract(image_path).lower().replace('\n', '').replace('!', '').replace('?', '').replace('.', ''))

print('score tesseract:', score_tesseract / 100)
print('score_easyocr:', score_easyocr / 100)
print('score_textract:', score_textract / 100)